In [ ]:
!pip install ultralytics

In [ ]:
!pip install easyocr

In [ ]:
# !gdown https://github.com/abewley/sort.git

In [ ]:
# import sys

In [ ]:
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/Parking System')

In [ ]:
# sys.path.append('/content/sort')

In [3]:
from ultralytics import YOLO
import os
import cv2
import easyocr
import string
# import sort
# import util

In [4]:
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [22]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Parking System/Testing/Image test.jpg'
file_path_out = '/content/drive/MyDrive/Colab Notebooks/Parking System/Output Testing/out'
file_out = '{}_out'.format(file_path_out)

In [29]:
_, file_ext = os.path.splitext(file_path)

if file_ext.lower() == '.jpg':
  frame = cv2.imread(file_path)
  ret, frame = frame.read()
  H, W, _ = frame.shape
else:
  cap = cv2.VideoCapture(file_path)
  ret, frame = cap.read()
  H, W, _ = frame.shape
  out = cv2.VideoWriter(file_out, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

AttributeError: ignored

In [24]:
model_path = '/content/drive/MyDrive/Colab Notebooks/Parking System/output model/100epoch_91%acc_NewDataset.pt'

In [25]:
model = YOLO(model_path)

In [26]:
results = {}
# mot_tracker = sort.Sort()

In [27]:
threshold = 0.1

while ret:
    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Extract text from bounding box using EasyOCR
            box = frame[int(y1):int(y2), int(x1):int(x2)]
            ocr_result = reader.readtext(box)
            text = ' '.join([line[1] for line in ocr_result])

            # Draw bounding box and text
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, text, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import csv
import cv2
import easyocr
import numpy as np
import os
from ultralytics import YOLO

# Inisialisasi model YOLOv8
model = YOLO('/content/Content/MyDrive/Project/DatasetRoboFlow/last.pt')

# Inisialisasi EasyOCR
reader = easyocr.Reader(['en'])

# Membuka video input
input_video_path = '/content/Content/MyDrive/Project/Dataset/TesVideo2/TesNewDataset.mp4'
cap = cv2.VideoCapture(input_video_path)
kernel_sharpening = np.array([[0,  1,  0],[1, -4,  1],[0,  1,  0]])

# Mendapatkan direktori video
video_directory = os.path.dirname(input_video_path)

# Membuat nama file CSV untuk menyimpan hasil OCR
csv_file_path = os.path.join(video_directory, 'ocr_results4.csv')
csv_file = open(csv_file_path, 'w', newline='')
csv_writer = csv.writer(csv_file)

# Menyimpan video output
output_video_path = '/content/Content/MyDrive/Project/Dataset/TesVideo2/NewDatasetOutput_video4.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Format kodek video
output_video = cv2.VideoWriter(output_video_path, fourcc, 20.0, (1280, 720))  # Resolusi sesuai dengan video input

threshold = 0.5  # Ambang batas untuk deteksi objek

while True:
    # Membaca frame dari video input
    ret, frame = cap.read()

    if not ret:
        break

    # Mendeteksi plat nomor motor dalam frame
    results = model(frame)[0]

    for result in results.boxes.data.tolist():
      x1, y1, x2, y2, score, class_id = result

      if score > threshold:
          # Extract text from bounding box using EasyOCR
          box = frame[int(y1):int(y2), int(x1):int(x2)]

          # Image Enhancement: Pra-pemrosesan
          resized_box = cv2.resize(box, None, fx=4, fy=4)  # Perbesar gambar
          sharpened_box = cv2.filter2D(resized_box, -1, kernel_sharpening)  # Sharpening
          denoised_box = cv2.fastNlMeansDenoisingColored(sharpened_box, None, 10, 10, 7, 21)  # Denoising

          # OCR pada citra yang telah ditingkatkan
          ocr_result = reader.readtext(denoised_box)
          text = ' '.join([line[1] for line in ocr_result])

          # Draw bounding box on the original frame
          cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)

          # Menyesuaikan ukuran denoised_box
          denoised_box = cv2.resize(denoised_box, (w, frame.shape[0]))

          # Menentukan koordinat pojok kanan atas
          x_offset = frame.shape[1] - w
          y_offset = 0

          # Menempatkan citra hasil crop yang sudah diubah ukuran di pojok kanan atas
          frame[y_offset:y_offset+h, x_offset:x_offset+w] = denoised_box


          # Menambahkan teks hasil OCR di atas citra
          cv2.putText(frame, text, (x_offset, y_offset - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

          # Menyimpan hasil OCR ke file CSV
          csv_writer.writerow([text])

      # Menyimpan frame yang telah dimodifikasi ke dalam video output
      output_video.write(frame)

# Tutup video input, output, dan file CSV
cap.release()
output_video.release()
csv_file.close()
cv2.destroyAllWindows()